# Deplot notebook
> Creating an interactive environment for comparing stocks 



In [ ]:
#| default_exp deploy_project

In [ ]:
#| export
import buy_and_hold_vs_arima
import yfinance as yf
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc


In [ ]:
from buy_and_hold_vs_arima.arima_stats import create_auto_arima_prediction_future_2
from buy_and_hold_vs_arima.arima_stats import create_dataframe_with_series

In [ ]:
#| hide 
# how to import packages ! 
from buy_and_hold_vs_arima import arima_stats
arima_stats.foo("barak")

'Hello HAHAHAHbarak'

In [ ]:
#help(buy_and_hold_vs_arima)
#buy_and_hold_vs_arima.foo("hba")
dir(buy_and_hold_vs_arima)
#dir(np)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'arima_stats']

In [ ]:
#| export 


companies = ['AMZN','NFLX','GOOG']
tickers = [yf.Ticker(ticker).history( start='2021-12-10', end='2022-12-30')['High'].rename(ticker) for ticker in companies]
df = pd.concat(tickers, axis=1)
df

for a in df.columns:
    print(a)
    
df[['AMZN','GOOG']]

AMZN
NFLX
GOOG


,AMZN,GOOG
Date,,
2021-12-10 00:00:00-05:00,175.927002,149.399994
2021-12-13 00:00:00-05:00,172.100006,148.562500
2021-12-14 00:00:00-05:00,169.498993,145.442001
2021-12-15 00:00:00-05:00,173.600006,147.517242
2021-12-16 00:00:00-05:00,174.166000,148.551498
...,...,...
2022-12-22 00:00:00-05:00,85.680000,89.180000
2022-12-23 00:00:00-05:00,85.779999,90.099998
2022-12-27 00:00:00-05:00,85.349998,89.500000


In [ ]:
#| export

# http://127.0.0.1:8050/

#df = px.data.gapminder()
#df.columns



external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dpdn2",
            value=["AMZN", "GOOG"],
            multi=True,
            options=[{"label": x, "value": x} for x in df.columns],
        ),
         dbc.Row(
            
               # dcc.Graph(id="pie-graph", figure={}, className="six columns"),
                dcc.Graph(
                    id="my-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
           ),
            dbc.Row(
                dcc.Graph(
                    id="basic-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
            ),
        dbc.Row(
            # Values is the initial values
            # first arg and second arg are the overall range , third is the jumps 
                dcc.RangeSlider(0, len(df) - 1 , 1, count=1, value=[0, len(df) - 1] , id="range-inference") 
        ),
        dbc.Row(
            dcc.Graph(
                    id="inference-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
        ),
        
        
    ]
)


@app.callback(
    Output(component_id="my-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    print(country_chosen)
    
    dff = df[country_chosen]
    predication_data = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country]).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
   
    fig = px.line(
        predication_data,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig

@app.callback(
    Output(component_id="inference-graph", component_property="figure"),
    Input('range-inference', 'value'),
    Input(component_id="dpdn2", component_property="value"),
)
def update_output(slider_value,country_chosen):
    #print(value)
    dff = df[country_chosen]
    first_date_infer = dff.index[slider_value[0]]
    last_date_infer = dff.index[slider_value[1]]
    predication = pd.DataFrame()
    #concated_data = dff[country][first_date_infer:last_date_infer].copy()
    predication = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country][first_date_infer:last_date_infer].copy()).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
    #predication.drop(country_chosen, axis=1)
    for country in country_chosen:
        predication[country + "True"] = dff[country][last_date_infer:]
    fig_global = px.line(
        predication,
        #x="Dates",
        #y=["Global Minimum Reg", "High"],
        #hover_data={"Dates": "|%B %d, %Y"},
        title="Inference BackTester",
    )

    fig_global.add_vline(
        x=first_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=first_date_infer, text=str(first_date_infer))

    fig_global.add_vline(
        x=last_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=last_date_infer, text=str(last_date_infer))
    
    
    
    
    
    return fig_global


@app.callback(
    Output(component_id="basic-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    
    dff = df[country_chosen]
    fig = px.line(
        dff,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig






In [ ]:
#| hide
#if __name__ == "__main__":
#    app.run_server(debug=False)


In [ ]:
#| export
if __name__ == "__main__":
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [12/Dec/2023 13:12:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2023 13:12:39] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



['AMZN', 'GOOG']


/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likeli

 ARIMA(1,0,1)(0,0,0)[0] intercept
 ARIMA(1,0,1)(0,0,0)[0] intercept


...

 ARIMA(1,0,0)(0,0,0)[0] intercept


/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: Future

 ARIMA(1,0,1)(0,0,0)[0] intercept


/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: 

 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept


In [ ]:
import nbdev; nbdev.nbdev_export()